In [5]:
import nltk
# nltk.download('punkt', download_dir='NLTK_DATA')
# nltk.download('stopwords', download_dir='NLTK_DATA')
nltk.data.path.append("NLTK_DATA")
import pandas as pd
import numpy as np
from processing import freqs, normalisasi
from sentiment import *

# PREPARASI *TRAINING DATASET*

In [6]:
# TODO: pindah ke sqlite
dataset = pd.read_csv("dataset/training_dataset.csv")
all_pos = dataset[dataset['label'] == 1]
all_neg = dataset[dataset['label'] == 0]

# * Training data: 80% pertama
# * Testing data: 20% terakhir
train_pos = all_pos[:int(len(all_pos)*0.8)]
train_neg = all_neg[:int(len(all_neg)*0.8)]
test_pos = all_pos[-int(len(all_pos)*0.2):]
test_neg = all_neg[-int(len(all_neg)*0.2):]

train_x = train_pos.append(train_neg, ignore_index=True)
test_x = test_pos.append(test_neg, ignore_index=True)

train_y = np.append(np.ones(len(train_pos)), np.zeros(len(train_neg)))
test_y = np.append(np.ones(len(test_pos)), np.zeros(len(test_neg)))

In [7]:
# ld = len(dataset)
# trp = len(train_pos)
# tng = len(train_neg)
# tsp = len(test_pos)
# tsn = len(test_neg)
# print(f"dataset: {ld} \n train pos: {trp} \n train neg: {tng} \n test pos: {tsp} \n test neg: {tsn}")

# PELATIHAN *MODEL CLASSIFIER*

In [9]:
x = train_x['text']
y = train_y
kamus_frek = kamus_freq(x, y)

In [13]:
kamus_frek

{('sabar', 1.0): 6,
 ('yaa', 1.0): 8,
 ('insya', 1.0): 3,
 ('allah', 1.0): 25,
 ('buka', 1.0): 7,
 ('pintu', 1.0): 1,
 ('syurga', 1.0): 1,
 ('halang', 1.0): 1,
 ('api', 1.0): 1,
 ('neraka', 1.0): 1,
 ('orang', 1.0): 26,
 ('tua', 1.0): 5,
 ('lagu', 1.0): 10,
 ('mbak', 1.0): 19,
 ('tata', 1.0): 3,
 ('kisah', 1.0): 4,
 ('nyata', 1.0): 5,
 ('rumah', 1.0): 7,
 ('tangga', 1.0): 2,
 ('bener', 1.0): 4,
 ('sayang', 1.0): 12,
 ('get', 1.0): 1,
 ('suami', 1.0): 18,
 ('syukur', 1.0): 7,
 ('moga', 1.0): 54,
 ('awet', 1.0): 2,
 ('komen', 1.0): 4,
 ('si', 1.0): 10,
 ('ga', 1.0): 43,
 ('sedih', 1.0): 5,
 ('ketawa', 1.0): 2,
 ('ya', 1.0): 46,
 ('iya', 1.0): 3,
 ('klo', 1.0): 20,
 ('dah', 1.0): 4,
 ('sakit', 1.0): 4,
 ('sm', 1.0): 13,
 ('pasang', 1.0): 9,
 ('smpe', 1.0): 2,
 ('berlarut', 1.0): 1,
 ('ngapain', 1.0): 1,
 ('sih', 1.0): 9,
 ('ampe', 1.0): 1,
 ('depresi', 1.0): 1,
 ('serahin', 1.0): 1,
 ('aja', 1.0): 28,
 ('bawa', 1.0): 3,
 ('happy', 1.0): 5,
 ('mski', 1.0): 1,
 ('mba', 1.0): 15,
 ('senyum',

In [12]:
for x in kamus_frek.keys():
    print(kamus_frek.get(x, 0))

6
8
3
25
7
1
1
1
1
1
26
5
10
19
3
4
5
7
2
4
12
1
18
7
54
2
4
10
43
5
2
46
3
20
4
4
13
9
2
1
1
9
1
1
1
28
3
5
1
15
6
6
10
18
8
5
17
1
1
9
1
2
8
17
6
2
1
6
3
30
51
4
1
27
1
2
1
4
9
2
2
2
42
1
3
1
2
1
2
1
1
1
6
1
7
6
7
2
2
1
8
1
9
3
20
9
1
1
5
1
4
3
1
1
1
1
6
1
1
2
1
13
2
3
4
13
1
2
1
1
1
5
17
1
5
1
1
4
5
2
2
1
8
8
14
1
1
1
14
1
1
2
43
4
14
2
13
2
6
3
24
1
4
1
6
12
3
1
7
1
10
1
26
4
2
2
5
3
1
1
2
2
1
2
2
2
1
1
1
18
10
1
1
2
8
8
3
3
1
5
9
2
4
3
2
1
1
2
1
2
1
1
1
6
3
8
1
2
8
1
49
2
4
1
1
16
2
17
1
2
1
4
1
1
1
2
11
1
8
1
3
1
1
1
1
21
1
1
1
1
3
2
6
11
2
13
2
1
1
5
1
1
4
1
1
1
36
1
1
1
1
1
1
2
1
1
2
2
2
3
4
20
1
2
3
3
1
4
4
3
2
6
2
1
4
1
7
20
1
3
2
1
1
9
3
2
4
3
2
6
11
8
1
9
1
1
1
4
1
2
2
2
6
4
5
4
1
1
21
3
2
1
1
1
14
2
2
23
1
2
7
1
1
12
17
5
1
1
1
7
4
2
1
3
1
2
1
2
2
3
4
6
1
11
1
1
1
1
2
4
1
1
1
6
3
7
4
1
1
2
3
1
1
1
1
8
2
1
1
1
1
1
1
8
1
8
2
3
1
1
2
2
2
1
1
10
2
1
9
1
1
1
3
4
1
1
1
2
1
1
1
1
1
40
1
1
2
4
2
5
1
1
1
1
1
5
16
1
1
3
2
2
1
1
4
6
5
1
2
1
3
11
1
1
1
2
3
1
1
1
9
1
2
6
2
7
1
1
12
34


## logprior
$D$ = jumlah dokumen <br>
$D_{pos}$ = jumlah dokumen positif <br>
$D_{neg}$ = jumlah dokumen negatif <br>
### probabilitas dokumen positif
$P(D_{pos}) = \frac{D_{pos}}{D}$
### probabilitas dokumen negatif
$P(D_{neg}) = \frac{D_{neg}}{D}$
### probabilitas prior
${logprior} = log \left( \frac{P(D_{pos})}{P(D_{neg})} \right) = log \left( \frac{D_{pos}}{D_{neg}} \right)$
<br> disederhanakan, menjadi: <br>
$logprior = P(D_{pos}) - P(D_{neg})$

## loglikelihood
$W$ = probabilitas positif/negatif kata <br>
$freq$ = frekuensi kata unik yang bernilai positif/negatif <br>
$N$ =  jumlah total kata positif/negatif <br>
$V$ = jumlah total kata unik (panjang dokumen frekuensi) <br>
### probabilitas kata positif
$P(W_{pos}) = \frac{freq_{pos} + 1}{N_{pos} + V}$
### probabilitas kata negatif
$P(W_{neg}) = \frac{freq_{neg} + 1}{N_{neg} + V}$
### likelihood kata
$\text{loglikelihood} = \log \left(\frac{P(W_{pos})}{P(W_{neg})} \right)$

In [ ]:
logprior, loglikelihood = train_nbc(kamus_frek, train_y)

In [ ]:
# file = open("/home/gaz/dev/flask-sk/database/prior.txt", "w")
# str = repr(logprior)
# file.write(str)
# file.close()

In [ ]:
# like = pd.DataFrame(list(loglikelihood.items()), columns = ['kata', 'likelihood'])
# like.to_csv('/home/gaz/dev/flask-sk/database/loglikelihood.csv')

## HASIL PENGUJIAN MODEL
- data *training*
- data *testing*

In [ ]:
# test = test_nbc(test_x['text'], test_y, logprior, loglikelihood)
# test

## UJI KESELURUHAN *DATASET*

In [ ]:
# data_test = test_nbc(dataset['text'], dataset['label'], logprior, loglikelihood)
# data_test

## *CONFUSION MATRIX*

In [ ]:
pdc = pd.DataFrame()
pdc = dataset
pdc['label_prediksi'] = pdc['text'].apply(lambda x: predict_nbc(x, logprior, loglikelihood))

In [ ]:
# TODO: Do more testing with thresholds values from 0, 0.5, 0.8, 0.9, 1
# * to note;
# treshold = -0.03272669350255697
# * default treshold is 0
def toLabel(x):
   if x > 0:
       return 1
   elif x < 0:
       return 0
   else:
       return "n"
pdc['prediksi'] = pdc['label_prediksi'].apply(lambda x: toLabel(x))

In [ ]:
tr = logprior
def toLabel(x):
   if x > tr:
       return 1
   elif x < tr:
       return 0
   else:
       return "n"
pdc['prediksi_tr'] = pdc['label_prediksi'].apply(lambda x: toLabel(x))

In [ ]:
conf_matrix = pd.crosstab(pdc['label'], pdc['prediksi'], rownames=['Actual'], colnames=['Predicted'])
conf_matrix

In [ ]:
conf2 = pd.crosstab(pdc['label'], pdc['prediksi_tr'], rownames=['Actual'], colnames=['Predicted'])
conf2

In [ ]:
import seaborn as sn
import matplotlib.pyplot as plt

sn.heatmap(conf_matrix, annot=True, fmt='g')
plt.show()

In [ ]:
sn.heatmap(conf2, annot=True, fmt='g')
plt.show()

# UJI CLASSIFIER PADA ARTIKEL BERITA

In [ ]:
berita = open('berita.txt', 'r')
berita = berita.read()

In [ ]:
beritanormal = normalisasi(berita)
frekberita = freqs(beritanormal)
# frekberita

## SENTIMEN ARTIKEL KESELURUHAN

In [ ]:
predict_nbc(berita, logprior, loglikelihood)

## SENTIMEN ARTIKEL PER_KALIMAT

In [ ]:
from nltk.tokenize import sent_tokenize
x = pd.DataFrame()
kalimat = sent_tokenize(berita)
x['kalimat'] = kalimat
x['kalimat_normal'] = x['kalimat'].apply(lambda x: normalisasi(x))

In [ ]:
x['label_prediksi'] = x['kalimat'].apply(lambda x: predict_nbc(x, logprior, loglikelihood))

In [ ]:
x

## SENTIMEN ARTIKEL PER_KATA

In [ ]:
berita_normal = normalisasi(berita)

In [ ]:
# * predict_nbc per-kata
def predict_perkata(text, logpri, loglik):
    l = []
    for w in text:
        x = predict_nbc(w, logpri, loglik)
        l.append({
            w,
            x
            })
    return l

In [ ]:
predict_perkata(berita_normal, logprior, loglikelihood)